In [1]:
import numpy as np
import pandas as pd
import os
from multiprocessing import Pool

### Get stock name and ticker name

In [2]:
def get_stock_name(df):
    stock_name = df.iloc[0, 0].split("|")[0].strip()
    ticker_name = df.iloc[3, 0].split(".")[0].strip()
    return stock_name, ticker_name

### Get all the rows containing date and stock price

In [4]:
def get_time_series_data(df, stock_name):
    # Get the index where we find Exchange Date value
    date_index = df[df.iloc[:, 0] == 'Exchange Date'].index

    # Remove all the rows before exchange date
    df = df.iloc[date_index[0]:].reset_index(drop=True)
    df = df.iloc[:, 0:10]
    
    # Make the first row the header
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    
    # Convert Exchange Date into date dtype
    df['Exchange Date'] = pd.to_datetime(df['Exchange Date'], format="%Y-%m-%d %H:%M:%S")
    
    # Order the time series to ascending
    df = df.iloc[::-1].reset_index(drop=True)
    
    # Rename column
    df = df[['Exchange Date', 'Close', 'Open', 'Low', 'High',
       'Volume', 'Turnover - USD', 'Flow']]
    # df = df.rename(columns={"Close": "Stock Price"})

    # Add stock name
    df['Stock Name'] = stock_name
    return df

### Load the data and write the clean data in separate folder

In [5]:
# def process_excel_file(file_path):
#     df = pd.read_excel(file_path, header=None)
#     stock_name, ticker_name = get_stock_name(df)
#     df_clean = get_time_series_data(df, stock_name)
#     output_file_path = os.path.join("Output_Data/Processed_Data_Step1", f"{stock_name}.csv")
#     df_clean.to_csv(output_file_path, index=False)
#     return df

# def read_excel_files_from_directory(directory):
#     # List all files in the directory
#     files = os.listdir(directory)
    
#     # Filter the list to include only Excel files
#     excel_files = [file for file in files if file.endswith('.xls') or file.endswith('.xlsx')]
    
#     # Create the full file paths
#     excel_file_paths = [os.path.join(directory, file) for file in excel_files]
    
#     # Use multiprocessing Pool to process files in parallel
#     with Pool() as pool:
#         dataframes = pool.map(process_excel_file, excel_file_paths)
    
#     return dataframes


# # Example usage
# directory = 'Input_Data'  # Replace with your directory path
# all_dataframes = read_excel_files_from_directory(directory)

# # Optional: Do something with all_dataframes if needed
# print(all_dataframes)

In [7]:
def read_excel_files_from_directory(directory):
    # List all files in the directory
    files = os.listdir(directory)
    
    # Filter the list to include only Excel files
    excel_files = [file for file in files if file.endswith('.xls') or file.endswith('.xlsx')]
    
    # Initialize a list to store DataFrames
    dataframes = []
    
    # Read each Excel file into a pandas DataFrame
    for file in excel_files:
        file_path = os.path.join(directory, file)
        df = pd.read_excel(file_path, header=None)
        stock_name, ticker_name = get_stock_name(df)
        df_clean = get_time_series_data(df, stock_name)
        df_clean.to_csv(f"../Input_Data/Processed_Files_Step1/{stock_name}.csv",index=False)
        dataframes.append(df_clean)
        
    return dataframes

# Example usage
directory = '../Input_Data/Raw_Files'  # Replace with your directory path
all_dataframes = read_excel_files_from_directory(directory)

In [8]:
# Optionally, combine all DataFrames into a single DataFrame
combined_df = pd.concat(all_dataframes, ignore_index=True)

display(combined_df)

,Exchange Date,Close,Open,Low,High,Volume,Turnover - USD,Flow,Stock Name
0,2008-01-02,17.146366,17.338552,16.959685,17.451161,86136960,NaN,0,Alphabet Inc
1,2008-01-03,17.149869,17.148118,16.929406,17.187906,65056920,NaN,0,Alphabet Inc
2,2008-01-04,16.440932,17.008732,16.390884,17.040513,107218180,NaN,0,Alphabet Inc
3,2008-01-07,16.246994,16.364358,15.949206,16.57306,128098900,NaN,0,Alphabet Inc
4,2008-01-08,15.807316,16.340835,15.790302,16.515004,106838980,NaN,0,Alphabet Inc
...,...,...,...,...,...,...,...,...,...
84880,2024-03-22,114.61,114.65,112.4,115.23,2236828,254948735,34087742396.788601,Enphase Energy Inc
84881,2024-03-25,111.05,114.01,110.945,116.31,2802683,313816711,33773925685.788601,Enphase Energy Inc
84882,2024-03-26,109.35,112.73,109.25,113.45,2551274,282313553,33491612132.788601,Enphase Energy Inc
84883,2024-03-27,119.8,110.7,110.7,120.08,4947099,581638740,34073250872.788601,Enphase Energy Inc


In [10]:
combined_df.to_csv("../Output_Data/cleaned_data.csv", index=False)